In [ ]:
import os, requests, zipfile, re
from collections import Counter

import numpy as np

In [ ]:
dataset_url = "https://github.com/entropicemergence/tiny_llm_server/releases/download/v0.1.0/TinyStoriesV2-GPT4-small.zip"
datset_folder = "stories"
os.makedirs(datset_folder, exist_ok=True)

def download_dataset(url):
    filename = url.split("/")[-1]
    filepath = os.path.join(datset_folder, filename)
    if not os.path.exists(filepath):
        response = requests.get(url)
        with open(filepath, "wb") as f:
            f.write(response.content)
download_dataset(dataset_url)

In [ ]:
filepath = "lec2/stories/TinyStoriesV2-GPT4-small.zip"
with zipfile.ZipFile(filepath, "r") as zip_ref:
    zip_ref.extractall(datset_folder)
with open("lec2/stories/TinyStoriesV2-GPT4-small.txt", "r", encoding="utf-8") as f:
    stories_text = f.read()

In [ ]:
# print (stories_text[:10000])
text_split = stories_text.split("<|endoftext|>")

print (len(text_split))
for split in text_split[:5]:
    print(split)
    print("-"*100)

In [ ]:
a1= np.array([1,2,3])
a2= np.array([1.1,2.1,3.1])


c1= np.array([-1,-1.5,0.2])

cosine_similarity_a = np.dot(a1,a2) / (np.linalg.norm(a) * np.linalg.norm(b))
print(cosine_similarity_a)

cosine_similarity_a_c1 = np.dot(a1,c1) / (np.linalg.norm(a1) * np.linalg.norm(c1))
print(cosine_similarity_a_c1)